## Advanced Lane Finding Project

The goals / steps of this project are the following:

* Compute the camera calibration matrix and distortion coefficients given a set of chessboard images.
* Apply a distortion correction to raw images.
* Use color transforms, gradients, etc., to create a thresholded binary image.
* Apply a perspective transform to rectify binary image ("birds-eye view").
* Detect lane pixels and fit to find the lane boundary.
* Determine the curvature of the lane and vehicle position with respect to center.
* Warp the detected lane boundaries back onto the original image.
* Output visual display of the lane boundaries and numerical estimation of lane curvature and vehicle position.

In [1]:
# import modules
import glob

from IPython.display import HTML
import cv2
import matplotlib.pyplot as plt
from moviepy.editor import VideoFileClip
import numpy as np

%matplotlib qt

## Previous Project

In [2]:
def grayscale(img):
    """Applies the Grayscale transform
    This will return an image with only one color channel
    but NOTE: to see the returned image as grayscale
    (assuming your grayscaled image is called 'gray')
    you should call plt.imshow(gray, cmap='gray')"""
    return cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    # Or use BGR2GRAY if you read an image with cv2.imread()
    # return cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
def canny(img, low_threshold, high_threshold):
    """Applies the Canny transform"""
    return cv2.Canny(img, low_threshold, high_threshold)

def gaussian_blur(img, kernel_size):
    """Applies a Gaussian Noise kernel"""
    return cv2.GaussianBlur(img, (kernel_size, kernel_size), 0)

def region_of_interest(img, vertices):
    """
    Applies an image mask.
    
    Only keeps the region of the image defined by the polygon
    formed from `vertices`. The rest of the image is set to black.
    `vertices` should be a numpy array of integer points.
    """
    #defining a blank mask to start with
    mask = np.zeros_like(img)   
    
    #defining a 3 channel or 1 channel color to fill the mask with depending on the input image
    if len(img.shape) > 2:
        channel_count = img.shape[2]  # i.e. 3 or 4 depending on your image
        ignore_mask_color = (255,) * channel_count
    else:
        ignore_mask_color = 255
        
    #filling pixels inside the polygon defined by "vertices" with the fill color    
    cv2.fillPoly(mask, vertices, ignore_mask_color)
    
    #returning the image only where mask pixels are nonzero
    masked_image = cv2.bitwise_and(img, mask)
    return masked_image


def draw_lines(img, lines, color=[255, 0, 0], thickness=2):
    slope_threshold = 0.4    
    imshape = img.shape
    
    # Find out left lanes and right lanes
    
    left_lines, right_lines = [], []
    slope_left_lines, slope_right_lines = [], []
    b_left_lines, b_right_lines = [], []
    
    for ind, line in enumerate(lines):
        for x1, y1, x2, y2 in line:
            dx = x2 - x1
            if dx == 0: # to avoid infinite slope
                dx = 1
                
            dy = y2 - y1
            slope = dy/dx
            b = y1 - slope*x1
            
            # Filter out lines with wrong inclination
            if -slope_threshold < slope < slope_threshold:
                continue
                
            # Filter out lines with wrong bottom intersection
            xb = (imshape[0] - b)/slope
            if xb < 0 or xb > imshape[1]:
                continue
                
            # Group left and right lines
            if slope < 0:
                left_lines.append(line[0].tolist())
                slope_left_lines.append(slope)
                b_left_lines.append(b)
            else:
                right_lines.append(line[0].tolist())
                slope_right_lines.append(slope)
                b_right_lines.append(b)

    # Remove outliers based on slope and b values
    
    def remove_outliers(lines, slope_list, b_list):
        if len(lines) == 0:
            return []
        
        avg_slope = np.mean(slope_list)
        std_slope = np.std(slope_list)
        avg_b = np.mean(b_list)
        std_b = np.std(b_list)
        new_lines = []
        for slope, b, line in zip(slope_list, b_list, lines):
            if slope < avg_slope - std_slope or slope > avg_slope + std_slope:
                continue
                
            if b < avg_b - std_b or b > avg_b + std_b:
                continue
                
            new_lines.append(line)
                
        return new_lines
    
    left_lines = remove_outliers(left_lines, slope_left_lines, b_left_lines)
    right_lines = remove_outliers(right_lines, slope_right_lines, b_right_lines)

    # Form one line
    
    def form_one_line(lines, slope_list, b_list):
        if len(lines) == 0:
            return []
        
        y_list = [y1 for x1, y1, x2, y2 in lines]
        y_list.extend([y2 for x1, y1, x2, y2 in lines])
        top_y = min(y_list)
        avg_slope = np.mean(slope_list)
        avg_b = np.mean(b_list)
        top_x = int(round((top_y - avg_b)/avg_slope))
        bottom_x = int(round((imshape[0] - avg_b)/avg_slope))
        
        return [[bottom_x, imshape[0], top_x, top_y]]
    
    left_lines = form_one_line(left_lines, slope_left_lines, b_left_lines)
    right_lines = form_one_line(right_lines, slope_right_lines, b_right_lines)

    lines = left_lines
    lines.extend(right_lines)

    len_lines = len(lines)
    
    lines = np.array(lines).reshape(len_lines, 1, 4)

    for line in lines:
        for x1,y1,x2,y2 in line:
            cv2.line(img, (x1, y1), (x2, y2), color, thickness)

def hough_lines(img, rho, theta, threshold, min_line_len, max_line_gap):
    """
    `img` should be the output of a Canny transform.
        
    Returns an image with hough lines drawn.
    """
    lines = cv2.HoughLinesP(img, rho, theta, threshold, np.array([]), minLineLength=min_line_len, maxLineGap=max_line_gap)
    line_img = np.zeros((img.shape[0], img.shape[1], 3), dtype=np.uint8)
    draw_lines(line_img, lines, thickness=10)
    return line_img
    
# Python 3 has support for cool math symbols.

def weighted_img(img, initial_img, α=0.8, β=1., γ=0.):
    """
    `img` is the output of the hough_lines(), An image with lines drawn on it.
    Should be a blank image (all black) with lines drawn on it.
    
    `initial_img` should be the image before any processing.
    
    The result image is computed as follows:
    
    initial_img * α + img * β + γ
    NOTE: initial_img and img must be the same shape!
    """
    return cv2.addWeighted(initial_img, α, img, β, γ)

In [3]:
def process_image(image):
    # NOTE: The output you return should be a color image (3 channel) for processing video below
    # TODO: put your pipeline here,
    # you should return the final output (image where lines are drawn on lanes)

    # Step 1: convert the image to gray scale
    gray_image = grayscale(image)
    
    # Step 2: apply gaussian_blur to smooth the image
    blur_image = gaussian_blur(gray_image, 5)
    
    # Step 3: apply Canny edge detection algorithm
    edge_image = canny(blur_image, 50, 150)

    # Step 4: apply mask filter
    imshape = image.shape
    vertices = np.array([[(20,imshape[0]),(imshape[1]/2-50, imshape[0]/2+60), (imshape[1]/2+50, imshape[0]/2+60), (imshape[1]-20,imshape[0])]], dtype=np.int32)
    mask_image = region_of_interest(edge_image, vertices)

    # Step 5: find lines using Hough transform
    hough_image = hough_lines(mask_image, 2, np.pi/180, 20, 50, 30)
    
    # Merge detected lines to the original image
    merg_image = weighted_img(hough_image, image)

    result = merg_image

    return result

In [4]:
#clip1 = VideoFileClip("test_videos/project_video.mp4")
clip1 = VideoFileClip("test_videos/project_video.mp4").subclip(1,5)
white_output = 'test_videos/project_video_out.mp4'
white_clip = clip1.fl_image(process_image) #NOTE: this function expects color images!!
%time white_clip.write_videofile(white_output, audio=False)

[MoviePy] >>>> Building video test_videos/project_video_out.mp4
[MoviePy] Writing video test_videos/project_video_out.mp4


 99%|█████████▉| 100/101 [00:08<00:00,  8.37it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: test_videos/project_video_out.mp4 

CPU times: user 2.32 s, sys: 157 ms, total: 2.47 s
Wall time: 11.8 s


In [5]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(white_output))

## Step 1: Camera Calibration Using Chessboard Images

In [6]:
# prepare object points, like (0,0,0), (1,0,0), (2,0,0) ....,(6,5,0)
objp = np.zeros((6*9,3), np.float32)
objp[:,:2] = np.mgrid[0:9,0:6].T.reshape(-1,2)

# Arrays to store object points and image points from all the images.
objpoints = [] # 3d points in real world space
imgpoints = [] # 2d points in image plane.

# Make a list of calibration images
images = glob.glob('camera_cal/calibration??.jpg')
images.sort()

# Step through the list and search for chessboard corners
for fname in images[:]:
    print(fname)
    img = cv2.imread(fname)
    gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)

    # Find the chessboard corners
    ret, corners = cv2.findChessboardCorners(gray, (9,6),None)

    # If found, add object points, image points
    if ret == False:
        continue
        
    objpoints.append(objp)
    imgpoints.append(corners)

    # Draw and display the corners
    img = cv2.drawChessboardCorners(img, (9,6), corners, ret)

    fname_out = fname[:-4] + "_pts.jpg"
    print(fname_out)
    cv2.imwrite(fname_out, img)

camera_cal/calibration01.jpg
camera_cal/calibration02.jpg
camera_cal/calibration02_pts.jpg
camera_cal/calibration03.jpg
camera_cal/calibration03_pts.jpg
camera_cal/calibration04.jpg
camera_cal/calibration05.jpg
camera_cal/calibration06.jpg
camera_cal/calibration06_pts.jpg
camera_cal/calibration07.jpg
camera_cal/calibration07_pts.jpg
camera_cal/calibration08.jpg
camera_cal/calibration08_pts.jpg
camera_cal/calibration09.jpg
camera_cal/calibration09_pts.jpg
camera_cal/calibration10.jpg
camera_cal/calibration10_pts.jpg
camera_cal/calibration11.jpg
camera_cal/calibration11_pts.jpg
camera_cal/calibration12.jpg
camera_cal/calibration12_pts.jpg
camera_cal/calibration13.jpg
camera_cal/calibration13_pts.jpg
camera_cal/calibration14.jpg
camera_cal/calibration14_pts.jpg
camera_cal/calibration15.jpg
camera_cal/calibration15_pts.jpg
camera_cal/calibration16.jpg
camera_cal/calibration16_pts.jpg
camera_cal/calibration17.jpg
camera_cal/calibration17_pts.jpg
camera_cal/calibration18.jpg
camera_cal/calib

## Step 2: Undistort the images

In [7]:
# Make a list of original images
images = glob.glob('camera_cal/calibration??.jpg')
images.sort()

def get_calibrate(objpoints, imgpoints, img_shape):
    ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints, img_shape, None, None)
    return mtx, dist

def get_undistort(img, mtx, dist):
    dst = cv2.undistort(img, mtx, dist, None, mtx)
    return dst

img = cv2.imread(images[0])
mtx, dist = get_calibrate(objpoints, imgpoints, img.shape[1::-1])

for fname in images[:]:
    print(fname)
    img = cv2.imread(fname)
    #dst_img, dist_mtx, dist_coef = cal_undistort(img, objpoints, imgpoints)
    dst_img = get_undistort(img, mtx, dist)
    fname_out = fname[:-4] + "_und.jpg"
    cv2.imwrite(fname_out, dst_img)

camera_cal/calibration01.jpg
camera_cal/calibration02.jpg
camera_cal/calibration03.jpg
camera_cal/calibration04.jpg
camera_cal/calibration05.jpg
camera_cal/calibration06.jpg
camera_cal/calibration07.jpg
camera_cal/calibration08.jpg
camera_cal/calibration09.jpg
camera_cal/calibration10.jpg
camera_cal/calibration11.jpg
camera_cal/calibration12.jpg
camera_cal/calibration13.jpg
camera_cal/calibration14.jpg
camera_cal/calibration15.jpg
camera_cal/calibration16.jpg
camera_cal/calibration17.jpg
camera_cal/calibration18.jpg
camera_cal/calibration19.jpg
camera_cal/calibration20.jpg


## Step 3 : Create Thresholded Binary Image

In [8]:
def convert_thresh(img, sx_thresh=(20, 100), s_thresh=(170, 255), color="gray"):
    # Conver to HLS color space
    hls_img = cv2.cvtColor(img, cv2.COLOR_BGR2HLS)
    
    # Sobel x
    l_channel = hls_img[:, :, 1]
    sobelx = cv2.Sobel(l_channel, cv2.CV_64F, 1, 0)
    abs_sobelx = np.absolute(sobelx)
    scaled_sobel = np.uint8(255 * abs_sobelx / np.max(abs_sobelx))
    
    # Threshold x gradient
    sxbinary = np.zeros_like(scaled_sobel)
    sxbinary[(scaled_sobel >= sx_thresh[0]) & (scaled_sobel < sx_thresh[1])] = 1
    
    # Threshold color channel
    s_channel = hls_img[:, :, 2]
    s_binary = np.zeros_like(s_channel)
    s_binary[(s_channel >= s_thresh[0]) & (s_channel < s_thresh[1])] = 1
    
    # Stack each channel
    if color == "color":
        combined_binary = np.dstack((np.zeros_like(sxbinary), sxbinary, s_binary)) * 255
    else:
        tmp_binary = np.zeros_like(sxbinary)
        tmp_binary[(sxbinary == 1) | (s_binary == 1)] = 1
        tmp_binary = np.dstack((tmp_binary, tmp_binary, tmp_binary)) * 255
        combined_binary = cv2.cvtColor(tmp_binary, cv2.COLOR_BGR2GRAY)
    
    return combined_binary
    
fnameL = glob.glob("test_images/test?.jpg")
fnameL.extend(glob.glob("test_images/straight_lines?.jpg"))
fnameL.sort()

for fname in fnameL:
    print(fname)
    img = cv2.imread(fname)
    und_img = get_undistort(img, mtx, dist)
    thr_img = convert_thresh(und_img, color="color")

    fname_out = fname[:-4] + "_thres.jpg"
    cv2.imwrite(fname_out, thr_img)

test_images/straight_lines1.jpg
test_images/straight_lines2.jpg
test_images/test1.jpg
test_images/test2.jpg
test_images/test3.jpg
test_images/test4.jpg
test_images/test5.jpg
test_images/test6.jpg


## Step 4 : Perspective Transform

In [9]:
# Get list of undistorted images
fname = "test_images/straight_lines1.jpg"

# Read image
img = cv2.imread(fname)
und_img = get_undistort(img, mtx, dist)
thr_img = convert_thresh(und_img, color="gray")
    
img_size = thr_img.shape[1::-1]
    
# Find corners
corner1 = (200, img_size[1])
corner2 = (img_size[0]//2 - 57, img_size[1]//2+100)
corner3 = (img_size[0]//2 + 60, img_size[1]//2+100)
corner4 = (img_size[0] - 180, img_size[1])
src_pts = np.float32([corner1, corner2, corner3, corner4])

# Plot corner lines
ori_thr_img = cv2.cvtColor(thr_img, cv2.COLOR_GRAY2BGR)
cv2.line(ori_thr_img, corner1, corner2, (0, 0, 255), 3)
cv2.line(ori_thr_img, corner2, corner3, (0, 0, 255), 3)
cv2.line(ori_thr_img, corner3, corner4, (0, 0, 255), 3)

fname_out = fname[:-4] + "_corn.jpg"
cv2.imwrite(fname_out, ori_thr_img)

# Compute transformation matrix
offsetx = 300
dst_pts = np.float32([[offsetx, img_size[1]],
                      [offsetx, 0],
                      [img_size[0] - offsetx, 0],
                      [img_size[0] - offsetx, img_size[1]]])
M = cv2.getPerspectiveTransform(src_pts, dst_pts)
    
# Transform the original image
wrp_img = cv2.warpPerspective(thr_img, M, img_size)
    
fname_out = fname[:-4] + "_warp.jpg"
cv2.imwrite(fname_out, wrp_img)

True

## Step 5 : Find The Lane Boundary

In [10]:

def find_lane_boundary(img):
    img_shape = img.shape[1::-1]

    histogram = np.sum(img[img_shape[1]//2:, :], axis=0)

    midpoint = img_shape[0]//2
    leftx_base = np.argmax(histogram[:midpoint])
    rightx_base = np.argmax(histogram[midpoint:]) + midpoint

    out_img = np.dstack((img, img, img))

    # Hyperparameters

    nwindows = 9
    margin = 100
    minpix = 50

    window_height = np.int(img_shape[1]//nwindows)

    # Identify the x and y positions of all nonzero pixels in the image

    nonzero = img.nonzero()
    nonzeroy = np.array(nonzero[0])
    nonzerox = np.array(nonzero[1])

    leftx_current = leftx_base
    rightx_current = rightx_base

    left_lane_inds = []
    right_lane_inds = []

    for window in range(nwindows):
        # Identify window boundaries in x and y
        win_y_low = img_shape[1] - (window + 1) * window_height
        win_y_high = img_shape[1] - window * window_height
    
        win_xleft_low = leftx_current - margin
        win_xleft_high = leftx_current + margin
        win_xright_low = rightx_current - margin
        win_xright_high = rightx_current + margin
    
        # Draw the windows
        cv2.rectangle(out_img, (win_xleft_low, win_y_low), (win_xleft_high, win_y_high), (0, 255, 0), 2)
        cv2.rectangle(out_img, (win_xright_low, win_y_low), (win_xright_high, win_y_high), (0, 255, 0), 2)
    
        # Identify nonzero pixels within window
        good_left_inds = ((nonzeroy >= win_y_low) & (nonzeroy < win_y_high) &
                      (nonzerox >= win_xleft_low) & (nonzerox < win_xleft_high)).nonzero()[0]
        good_right_inds = ((nonzeroy >= win_y_low) & (nonzeroy < win_y_high) &
                       (nonzerox >= win_xright_low) & (nonzerox < win_xright_high)).nonzero()[0]
    
        # Append these indices to the lists
        left_lane_inds.append(good_left_inds)
        right_lane_inds.append(good_right_inds)
    
        # If there are more than minpix pixels, recenter the next window
        if len(good_left_inds) > minpix:
            leftx_current = np.int(np.mean(nonzerox[good_left_inds]))
        if len(good_right_inds) > minpix:
            rightx_current = np.int(np.mean(nonzerox[good_right_inds]))

    left_lane_inds = np.concatenate(left_lane_inds)
    right_lane_inds = np.concatenate(right_lane_inds)

    # Extract left and right line pixel positions
    leftx = nonzerox[left_lane_inds]
    lefty = nonzeroy[left_lane_inds]
    rightx = nonzerox[right_lane_inds]
    righty = nonzeroy[right_lane_inds]

    # Fit a second order polynomial
    left_fit = np.polyfit(lefty, leftx, 2)
    right_fit = np.polyfit(righty, rightx, 2)
    
    # Generate x and y for plotting
    ploty = np.linspace(0, img_shape[1] - 1, img_shape[1])
    left_fitx = left_fit[0] * ploty**2 + left_fit[1] * ploty + left_fit[2]
    right_fitx = right_fit[0] * ploty**2 + right_fit[1] * ploty + right_fit[2]

    # Plot left and right polynomial
    for y, x in zip(ploty, left_fitx):
        cv2.circle(out_img, (int(x), int(y)), 3, [0, 0, 255])
    for y, x in zip(ploty, right_fitx):
        cv2.circle(out_img, (int(x), int(y)), 3, [0, 0, 255])
    
    return left_fit, right_fit, out_img

fname = "test_images/straight_lines1_warp.jpg"
img = cv2.imread(fname, cv2.IMREAD_GRAYSCALE)
img_shape = img.shape[1::-1]

left_fit, right_fit, out_img = find_lane_boundary(img)

#out_img[lefty, leftx] = [255, 0, 0]
#out_img[righty, rightx] = [0, 0, 255]

fname_out = fname[:-4] + "_wind.jpg"
cv2.imwrite(fname_out, out_img)

True

In [19]:
def find_lane_boundary_with_line(img, left_fit, right_fit):
    img_shape = img.shape[1::-1]

#    histogram = np.sum(img[img_shape[1]//2:, :], axis=0)

#    midpoint = img_shape[0]//2
#    leftx_base = np.argmax(histogram[:midpoint])
#    rightx_base = np.argmax(histogram[midpoint:]) + midpoint

    out_img = np.dstack((img, img, img))

    # Hyperparameters

#    nwindows = 9
    margin = 30
#    minpix = 50

#    window_height = np.int(img_shape[1]//nwindows)

    # Identify the x and y positions of all nonzero pixels in the image

    nonzero = img.nonzero()
    nonzeroy = np.array(nonzero[0])
    nonzerox = np.array(nonzero[1])

#    leftx_current = leftx_base
#    rightx_current = rightx_base

    left_lane_inds = ((nonzerox > (left_fit[0]*(nonzeroy**2) + left_fit[1]*nonzeroy + 
                    left_fit[2] - margin)) & (nonzerox < (left_fit[0]*(nonzeroy**2) + 
                    left_fit[1]*nonzeroy + left_fit[2] + margin)))
    right_lane_inds = ((nonzerox > (right_fit[0]*(nonzeroy**2) + right_fit[1]*nonzeroy + 
                    right_fit[2] - margin)) & (nonzerox < (right_fit[0]*(nonzeroy**2) + 
                    right_fit[1]*nonzeroy + right_fit[2] + margin)))

    # Extract left and right line pixel positions
    leftx = nonzerox[left_lane_inds]
    lefty = nonzeroy[left_lane_inds]
    rightx = nonzerox[right_lane_inds]
    righty = nonzeroy[right_lane_inds]

    # Fit a second order polynomial
    left_fit = np.polyfit(lefty, leftx, 2)
    right_fit = np.polyfit(righty, rightx, 2)
    
    # Generate x and y for plotting
    ploty = np.linspace(0, img_shape[1] - 1, img_shape[1])
    left_fitx = left_fit[0] * ploty**2 + left_fit[1] * ploty + left_fit[2]
    right_fitx = right_fit[0] * ploty**2 + right_fit[1] * ploty + right_fit[2]

    # Plot left and right polynomial
    for y, x in zip(ploty, left_fitx):
        cv2.circle(out_img, (int(x), int(y)), 3, [0, 0, 255])
    for y, x in zip(ploty, right_fitx):
        cv2.circle(out_img, (int(x), int(y)), 3, [0, 0, 255])
    
    return left_fit, right_fit, out_img

fname = "test_images/straight_lines1_warp.jpg"
img = cv2.imread(fname, cv2.IMREAD_GRAYSCALE)
img_shape = img.shape[1::-1]

print(left_fit)
print(right_fit)
left_fit, right_fit, out_img = find_lane_boundary_with_line(img, left_fit, right_fit)
print(left_fit)
print(right_fit)

#out_img[lefty, leftx] = [255, 0, 0]
#out_img[righty, rightx] = [0, 0, 255]

fname_out = fname[:-4] + "_wind.jpg"
cv2.imwrite(fname_out, out_img)

[  5.41665804e-06   1.50090877e-02   2.94891682e+02]
[  3.37500984e-05  -2.82560519e-02   9.87256689e+02]
[ -3.86537296e-06   2.28646559e-02   2.93898569e+02]
[  2.65386877e-05  -2.40892315e-02   9.86853713e+02]


True

## Step 6 : Determine the curvature of the lane and vehicle position with respect to center

In [20]:
mx = 3.7/690
my = 30/720

def calculate_curvature(curve_fit, y, mx, my):
    fit0 = mx/(my**2)*curve_fit[0]
    fit1 = mx/my*curve_fit[1]
    fit2 = mx*curve_fit[2]
    
    curvature = ((1 + (2 * fit0 * y + fit1)**2)**1.5)/abs(2 * fit0)
    return curvature

def calculate_vehicle_position(left_fit, right_fit):
    pass

print(calculate_curvature(left_fit, img_shape[1], mx, my))
print(calculate_curvature(right_fit, img_shape[1], mx, my))

41892.4484881
6221.06736709


## Step 7 : Warp the detected lane boundaries back onto the original image.

In [21]:
def draw_lanes(img, left_fit, right_fit, M):
    lane_wrp_img = np.zeros_like(img)
    img_size = lane_wrp_img.shape[1::-1]
    
    # Generate x and y for plotting
    ploty = np.linspace(0, img_shape[1] - 1, img_shape[1])
    left_fitx = left_fit[0] * ploty**2 + left_fit[1] * ploty + left_fit[2]
    right_fitx = right_fit[0] * ploty**2 + right_fit[1] * ploty + right_fit[2]

    # Plot left and right polynomial
    alpha_mask = np.zeros_like(lane_wrp_img)
    for y, x in zip(ploty, left_fitx):
        cv2.circle(lane_wrp_img, (int(x), int(y)), 10, [255, 0, 0])
        cv2.circle(alpha_mask, (int(x), int(y)), 10, [255, 255, 255])
    for y, x in zip(ploty, right_fitx):
        cv2.circle(lane_wrp_img, (int(x), int(y)), 10, [255, 0, 0])
        cv2.circle(alpha_mask, (int(x), int(y)), 10, [255, 255, 255])

    lane_unw_img = cv2.warpPerspective(lane_wrp_img, M, img_shape, flags=cv2.WARP_INVERSE_MAP)
    alpha_mask = cv2.warpPerspective(alpha_mask, M, img_shape, flags=cv2.WARP_INVERSE_MAP)
    
    # now use transparent mask (alpha blending)
    # https://www.learnopencv.com/alpha-blending-using-opencv-cpp-python/
    
    alpha_mask = alpha_mask.astype(float)/255
    foreground = cv2.multiply(alpha_mask, lane_unw_img.astype(float))
    background = cv2.multiply(1.0 - alpha_mask, img.astype(float))
    out_img = cv2.add(foreground, background)
    
    return out_img

fname = "test_images/straight_lines1.jpg"
img = cv2.imread(fname)

und_img = get_undistort(img, mtx, dist)
thr_img = convert_thresh(und_img)
wrp_img = cv2.warpPerspective(thr_img, M, thr_img.shape[1::-1])
left_fit, right_fit, out_img = find_lane_boundary(wrp_img)
out_img = draw_lanes(img, left_fit, right_fit, M)

cv2.imwrite("test_images/straight_lines1_final.jpg", out_img)

True

## Step 8 : Output visual display of the lane boundaries and numerical estimation of lane curvature and vehicle position.

## Put everything together

In [22]:
def process_image_updated(img, left_fit=[None], right_fit=[None]):
    und_img = get_undistort(img, mtx, dist)
    thr_img = convert_thresh(und_img)
    wrp_img = cv2.warpPerspective(thr_img, M, thr_img.shape[1::-1])
    
    if left_fit == [None]:
        left_fit[0], right_fit[0], out_img = find_lane_boundary(wrp_img)
    else:
        left_fit[0], right_fit[0], out_img = find_lane_boundary_with_line(wrp_img, left_fit[0], right_fit[0])
        
    out_img = draw_lanes(img, left_fit[0], right_fit[0], M)

    return out_img

In [25]:
clip = VideoFileClip("test_videos/project_video.mp4")
#clip = VideoFileClip("test_videos/project_video.mp4").subclip(22, 25)
white_output = 'test_videos/project_video_out_updated.mp4'
white_clip = clip.fl_image(process_image_updated) #NOTE: this function expects color images!!
%time white_clip.write_videofile(white_output, audio=False)

[MoviePy] >>>> Building video test_videos/project_video_out_updated.mp4
[MoviePy] Writing video test_videos/project_video_out_updated.mp4


100%|█████████▉| 1260/1261 [07:07<00:00,  2.96it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: test_videos/project_video_out_updated.mp4 

CPU times: user 4min 9s, sys: 49 s, total: 4min 58s
Wall time: 7min 10s


In [26]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(white_output))

In [27]:
clip = VideoFileClip("test_videos/challenge_video.mp4")
white_output = 'test_videos/challenge_video_out_updated.mp4'
white_clip = clip.fl_image(process_image_updated) #NOTE: this function expects color images!!
%time white_clip.write_videofile(white_output, audio=False)

[MoviePy] >>>> Building video test_videos/challenge_video_out_updated.mp4
[MoviePy] Writing video test_videos/challenge_video_out_updated.mp4


100%|██████████| 485/485 [02:44<00:00,  2.86it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: test_videos/challenge_video_out_updated.mp4 

CPU times: user 1min 32s, sys: 24.8 s, total: 1min 57s
Wall time: 2min 47s


In [28]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(white_output))